In [2]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
data_path = "/content/drive/MyDrive/fp/train_set_final.parquet"

In [4]:
test_data_path = "/content/drive/MyDrive/fp/test_set_final.parquet"

In [5]:
%cd /content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/

/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT


In [6]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [7]:
import pandas as pd

In [8]:
!pip install fastNLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.7/644.7 kB 29.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.3/217.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 4.3 MB/s eta 0:00:00
  Created wheel for fastNLP: filename=FastNLP-1.0.1-py3-none-any.whl size=811350 sha256=422de6965d539c3a7b4cc33aa3992c69bb2bb9921fec75858c985bb56acf15db
  Stored in directory: /root/.cache/pip/wheels/d9/2a/3d/94e7148cb717edd3890429b716692c12460b2436b965bf999e
Successfully built fastNLP
  Attempting uninstall: rich
    Found existing installation: rich 13.9.4
    Uninstalling rich-13.9.4:
      Successfully uninstalled rich-13.9.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 11.2.0 which is incompatible.
pymc 5.21.1 requ

In [9]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

## create datasets

In [68]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split

# Load your CSV
df = pd.read_parquet(data_path)  # Update path if needed
df = df[['text','source']]
df = df.rename(columns = {'source':'label'})
df['label'] = df['label'].astype('str')
df

,text,label
371533,SAP is a big deal because it’s one of the lead...,1
475497,Creating a drug to make people lose weight is ...,1
212345,“Semantic satiation” is the phrase used to des...,0
287410,Celebrity nudes are typically leaked through h...,1
666934,"""@kirstiealley My little chihuahua and I are s...",1
...,...,...
193502,Any system that uses usernames and passwords h...,0
186701,We the OP means is if you were going to copy a...,0
33726,You are entering your first R.E.M. cycle. This...,0
410038,America has a massive cultural influence due t...,1


In [69]:
import pandas as pd
import json
import numpy as np

# Load your dataframe (assume df is already loaded)
# e.g., df = pd.read_csv("your_data.csv")

# Step 1: Define your label mapping
label_to_str = {'1': 'ai', '0': 'human'}
df['label_str'] = df['label'].map(label_to_str)  # <- string version of label

# Step 2: Add required dummy fields
df['label_int'] = df['label']  # keep 0/1 version if needed
df['prompt_len'] = 0
df['begin_idx_list'] = [[0, 0, 0, 0]] * len(df)

def make_dummy_ll_tokens():
    return [[0.0] * 10 for _ in range(4)]

df['ll_tokens_list'] = [make_dummy_ll_tokens() for _ in range(len(df))]

# Step 3: Sample your data
sampled_train = df.sample(n=5000, random_state=42)

# Step 4: Save to JSONL with correct string labels
def save_jsonl(df, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            obj = {
                "text": row['text'],
                "label": row['label_str'],  # ✅ <-- string version: 'ai' or 'human'
                "label_int": int(row['label_int']),  # optional: keep int for ref
                "prompt_len": int(row['prompt_len']),
                "begin_idx_list": row['begin_idx_list'],
                "ll_tokens_list": row['ll_tokens_list']
            }
            f.write(json.dumps(obj, ensure_ascii=False) + '\n')

# Save both files
save_jsonl(sampled_train, "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/train_final_seqx.jsonl")


In [70]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split

# Load your CSV
df = pd.read_parquet(test_data_path)  # Update path if needed
df = df[['text','source']]
df = df.rename(columns = {'source':'label'})
df['label'] = df['label'].astype('str')
df

,text,label
99236,We still have a dominant foot as well. Think a...,0
677686,"""Only 8 more days until I turn 20! My iPod see...",1
348424,The AC button controls the air conditioning co...,1
392131,"""Yup, you're right, it's crazy that there's st...",1
162682,The worst is after being out all day on rough ...,0
...,...,...
45509,"The ink doesn't dry, because the little ball a...",0
760836,What’s the best advice you’ve ever received? S...,1
620555,"@""tchvinkle thank you so much, bestie!""",1
283444,They ask for race and gender info mainly for r...,1


In [71]:
import pandas as pd
import json
import numpy as np

# Load your dataframe (assume df is already loaded)
# e.g., df = pd.read_csv("your_data.csv")

# Step 1: Define your label mapping
label_to_str = {'1': 'ai', '0': 'human'}
df['label_str'] = df['label'].map(label_to_str)  # <- string version of label

# Step 2: Add required dummy fields
df['label_int'] = df['label']  # keep 0/1 version if needed
df['prompt_len'] = 0
df['begin_idx_list'] = [[0, 0, 0, 0]] * len(df)

def make_dummy_ll_tokens():
    return [[0.0] * 10 for _ in range(4)]

df['ll_tokens_list'] = [make_dummy_ll_tokens() for _ in range(len(df))]

# Step 3: Sample your data
sampled_train = df.sample(n=2500, random_state=42)

# Step 4: Save to JSONL with correct string labels
def save_jsonl(df, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            obj = {
                "text": row['text'],
                "label": row['label_str'],  # ✅ <-- string version: 'ai' or 'human'
                "label_int": int(row['label_int']),  # optional: keep int for ref
                "prompt_len": int(row['prompt_len']),
                "begin_idx_list": row['begin_idx_list'],
                "ll_tokens_list": row['ll_tokens_list']
            }
            f.write(json.dumps(obj, ensure_ascii=False) + '\n')

# Save both files
save_jsonl(sampled_train, "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/test_final_seqx.jsonl")


In [72]:
!ls

dataloader.py				predictions_debug.json	test_mini_10.pt
model.py				__pycache__		train_final_seqx.jsonl
predictions_debug_20250401_032245.json	seqxgen_final.pt	train.py
predictions_debug_20250405_004749.json	test_final_seqx.jsonl


In [74]:
!python train.py \
  --train_path train_final_seqx.jsonl \
  --test_path test_final_seqx.jsonl \
  --model Transformer \
  --seq_len 512 \
  --batch_size 16 \
  --num_train_epochs 1 \
  --lr 5e-5 \
  --ckpt_name correct_data.pt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-04-05 18:10:37.340924: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743876637.360162   21064 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743876637.365086   21064 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-05 18:10:37.386951: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
100% 5000/5000

# Testing on full 50k/15k dataset

In [15]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split

# Load your CSV
df = pd.read_parquet(data_path)  # Update path if needed
df = df[['text','source']]
df = df.rename(columns = {'source':'label'})
df['label'] = df['label'].astype('str')
df

,text,label
371533,SAP is a big deal because it’s one of the lead...,1
475497,Creating a drug to make people lose weight is ...,1
212345,“Semantic satiation” is the phrase used to des...,0
287410,Celebrity nudes are typically leaked through h...,1
666934,"""@kirstiealley My little chihuahua and I are s...",1
...,...,...
193502,Any system that uses usernames and passwords h...,0
186701,We the OP means is if you were going to copy a...,0
33726,You are entering your first R.E.M. cycle. This...,0
410038,America has a massive cultural influence due t...,1


In [16]:
import pandas as pd
import json
import numpy as np

# Load your dataframe (assume df is already loaded)
# e.g., df = pd.read_csv("your_data.csv")

# Step 1: Define your label mapping
label_to_str = {'1': 'ai', '0': 'human'}
df['label_str'] = df['label'].map(label_to_str)  # <- string version of label

# Step 2: Add required dummy fields
df['label_int'] = df['label']  # keep 0/1 version if needed
df['prompt_len'] = 0
df['begin_idx_list'] = [[0, 0, 0, 0]] * len(df)

def make_dummy_ll_tokens():
    return [[0.0] * 10 for _ in range(4)]

df['ll_tokens_list'] = [make_dummy_ll_tokens() for _ in range(len(df))]

# Step 3: Sample your data
sampled_train = df.sample(n=30000, random_state=42)

# Step 4: Save to JSONL with correct string labels
def save_jsonl(df, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            obj = {
                "text": row['text'],
                "label": row['label_str'],  # ✅ <-- string version: 'ai' or 'human'
                "label_int": int(row['label_int']),  # optional: keep int for ref
                "prompt_len": int(row['prompt_len']),
                "begin_idx_list": row['begin_idx_list'],
                "ll_tokens_list": row['ll_tokens_list']
            }
            f.write(json.dumps(obj, ensure_ascii=False) + '\n')

# Save both files
save_jsonl(sampled_train, "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/full_train_final_seqx.jsonl")


In [17]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split

# Load your CSV
df = pd.read_parquet(test_data_path)  # Update path if needed
df = df[['text','source']]
df = df.rename(columns = {'source':'label'})
df['label'] = df['label'].astype('str')
df

,text,label
99236,We still have a dominant foot as well. Think a...,0
677686,"""Only 8 more days until I turn 20! My iPod see...",1
348424,The AC button controls the air conditioning co...,1
392131,"""Yup, you're right, it's crazy that there's st...",1
162682,The worst is after being out all day on rough ...,0
...,...,...
45509,"The ink doesn't dry, because the little ball a...",0
760836,What’s the best advice you’ve ever received? S...,1
620555,"@""tchvinkle thank you so much, bestie!""",1
283444,They ask for race and gender info mainly for r...,1


In [18]:
import pandas as pd
import json
import numpy as np

# Load your dataframe (assume df is already loaded)
# e.g., df = pd.read_csv("your_data.csv")

# Step 1: Define your label mapping
label_to_str = {'1': 'ai', '0': 'human'}
df['label_str'] = df['label'].map(label_to_str)  # <- string version of label

# Step 2: Add required dummy fields
df['label_int'] = df['label']  # keep 0/1 version if needed
df['prompt_len'] = 0
df['begin_idx_list'] = [[0, 0, 0, 0]] * len(df)

def make_dummy_ll_tokens():
    return [[0.0] * 10 for _ in range(4)]

df['ll_tokens_list'] = [make_dummy_ll_tokens() for _ in range(len(df))]

# Step 3: Sample your data
sampled_train = df.sample(n=10000, random_state=42)

# Step 4: Save to JSONL with correct string labels
def save_jsonl(df, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            obj = {
                "text": row['text'],
                "label": row['label_str'],  # ✅ <-- string version: 'ai' or 'human'
                "label_int": int(row['label_int']),  # optional: keep int for ref
                "prompt_len": int(row['prompt_len']),
                "begin_idx_list": row['begin_idx_list'],
                "ll_tokens_list": row['ll_tokens_list']
            }
            f.write(json.dumps(obj, ensure_ascii=False) + '\n')

# Save both files
save_jsonl(sampled_train, "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/full_test_final_seqx.jsonl")


In [19]:
!python train.py \
  --train_path full_train_final_seqx.jsonl \
  --test_path full_test_final_seqx.jsonl \
  --model Transformer \
  --seq_len 512 \
  --batch_size 16 \
  --num_train_epochs 1 \
  --lr 5e-5 \
  --ckpt_name correct_data.pt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-04-05 19:10:31.105260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743880231.147223   11179 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743880231.159054   11179 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-05 19:10:31.196358: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
100% 30000/300

# failed old runs

In [ ]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split

# Map labels to integers
unique_labels = sorted(df['label'].unique())
label_to_int = {label: idx for idx, label in enumerate(unique_labels)}
df['label_int'] = df['label'].map(label_to_int)

# Add required dummy fields
df['prompt_len'] = 0
df['begin_idx_list'] = [[0, 0, 0, 0]] * len(df)

# Create dummy ll_tokens_list (4 channels of 10-dim zero vectors)
def make_dummy_ll_tokens():
    return [[0.0] * 10 for _ in range(4)]

df['ll_tokens_list'] = [make_dummy_ll_tokens() for _ in range(len(df))]

sampled = df.sample(n = 50000, random_state=42)

# Save to JSONL
def save_jsonl(df, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            obj = {
                "text": row['text'],
                "label": row['label'],
                "label_int": int(row['label_int']),
                "prompt_len": int(row['prompt_len']),
                "begin_idx_list": row['begin_idx_list'],
                "ll_tokens_list": row['ll_tokens_list']
            }
            f.write(json.dumps(obj, ensure_ascii=False) + '\n')

save_jsonl(sampled, "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/train_final_seqx.json")

In [ ]:
!python train.py \
  --train_path train_final_seqx.json \
  --test_path tiny_test.json \
  --model Transformer \
  --seq_len 512 \
  --batch_size 16 \
  --num_train_epochs 1 \
  --lr 5e-5 \
  --ckpt_name correct_data.pt


In [6]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 11.2.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system ==

In [7]:
!pip install fastNLP

In [ ]:
import pandas as pd
import json

# Load the original CSV
df = pd.read_csv(data_path)

# Shuffle & sample just 10 total
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
tiny_train_df = df.iloc[:5]
tiny_test_df = df.iloc[5:10]

# Prepare dummy fields again
tiny_train_df['label_int'] = tiny_train_df['label'].astype('category').cat.codes
tiny_test_df['label_int'] = tiny_test_df['label'].astype('category').cat.codes

tiny_train_df['prompt_len'] = 0
tiny_test_df['prompt_len'] = 0
tiny_train_df['begin_idx_list'] = [[0, 0, 0, 0]] * len(tiny_train_df)
tiny_test_df['begin_idx_list'] = [[0, 0, 0, 0]] * len(tiny_test_df)

def make_dummy_ll_tokens():
    return [[0.0] * 10 for _ in range(4)]

tiny_train_df['ll_tokens_list'] = [make_dummy_ll_tokens() for _ in range(len(tiny_train_df))]
tiny_test_df['ll_tokens_list'] = [make_dummy_ll_tokens() for _ in range(len(tiny_test_df))]

# Save to JSONL
def save_jsonl(df, path):
    with open(path, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            f.write(json.dumps({
                "text": row['text'],
                "label": row['label'],
                "label_int": int(row['label_int']),
                "prompt_len": int(row['prompt_len']),
                "begin_idx_list": row['begin_idx_list'],
                "ll_tokens_list": row['ll_tokens_list']
            }, ensure_ascii=False) + '\n')

save_jsonl(tiny_train_df, "tiny_train.json")
save_jsonl(tiny_test_df, "tiny_test.json")
print("✅ 10-sample test file saved as seqxgpt_ready_data_10.json")


✅ 10-sample test file saved as seqxgpt_ready_data_10.json


<ipython-input-49-d58853014976>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tiny_train_df['label_int'] = tiny_train_df['label'].astype('category').cat.codes
<ipython-input-49-d58853014976>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tiny_test_df['label_int'] = tiny_test_df['label'].astype('category').cat.codes
<ipython-input-49-d58853014976>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
!python train.py \
  --train_path tiny_train.json \
  --test_path tiny_test.json \
  --model Transformer \
  --seq_len 512 \
  --batch_size 16 \
  --num_train_epochs 1 \
  --lr 5e-5 \
  --ckpt_name test_mini_10.pt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-03-27 02:14:26.713372: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743041666.749247   65631 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743041666.761038   65631 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-27 02:14:26.795826: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
100% 5/5 [00:0

In [ ]:
combined_path = "/content/drive/MyDrive/fp/combined_df_preprocessed.parquet"

In [ ]:
df=pd.read_parquet(combined_path)

In [ ]:
df.head()

,text,label
0,"Keep the defense honest, get a feel for the pa...",0
1,"As someone who uses quality Premium, I wish th...",0
2,It's a quirk of the human eye. At the center o...,0
3,I'm pretty confident most of it isn't going ba...,0
4,"Automatics weigh more, so that alone makes gas...",0


In [ ]:
sampled_df = df.sample(n=100000, random_state=42)

In [ ]:
# Save in seq2gen format to JSONL
with open("seq2gen_ready_data_100k.jsonl", "w", encoding="utf-8") as f:
    for _, row in sampled_df.iterrows():
        json.dump({"input": row["text"], "target": row["label"]}, f, ensure_ascii=False)
        f.write("\n")

In [ ]:
data_path = "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/seq2gen_ready_data_100k.jsonl"

In [ ]:
df

,input,target
0,I'm pretty sure this is because after you take...,0
1,Whales and other deep-diving animals have evol...,1
2,The defeated deal with the EU means that Brexi...,1
3,Satellites don’t often collide because there’s...,1
4,"""Both sides will need to change."" Times report...",0
...,...,...
99995,@ohhaikelly awwww you're so nice why can't th...,0
99996,"When a car accelerates, the force of accelerat...",1
99997,"36deg celsius, pani puri, my room..all getting...",0
99998,They rely on simple rules and cues from their ...,1


In [ ]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split

# Load your JSONL
temp_df = pd.read_json(data_path, lines=True)
df = pd.DataFrame()
df['text'] = temp_df['input']
df['label'] = temp_df['target']

# Map labels to integers
unique_labels = sorted(df['label'].dropna().unique())
label_to_int = {label: idx for idx, label in enumerate(unique_labels)}
df['label_int'] = df['label'].map(label_to_int)

# Add required dummy fields
df['prompt_len'] = 0
df['begin_idx_list'] = [[0, 0, 0, 0]] * len(df)

# Create dummy ll_tokens_list (4 channels of 10-dim zero vectors)
def make_dummy_ll_tokens():
    return [[0.0] * 10 for _ in range(4)]

df['ll_tokens_list'] = [make_dummy_ll_tokens() for _ in range(len(df))]

# Split into train and test
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Save to JSONL
def save_jsonl(df, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            obj = {
                "text": row['text'],
                "label": row['label'],
                "label_int": int(row['label_int']),
                "prompt_len": int(row['prompt_len']),
                "begin_idx_list": row['begin_idx_list'],
                "ll_tokens_list": row['ll_tokens_list']
            }
            f.write(json.dumps(obj, ensure_ascii=False) + '\n')

save_jsonl(train_df, "train.json")
save_jsonl(test_df, "test.json")

print("✅ Finished writing train.json and test.json")


✅ Finished writing train.json and test.json


In [ ]:
!wc -l test.json

10000 test.json


In [ ]:
!python train.py \
  --train_path train.json \
  --test_path test.json \
  --model Transformer \
  --seq_len 512 \
  --batch_size 8 \
  --num_train_epochs 5 \
  --lr 5e-5 \
  --ckpt_name seqxgen_final.pt


Streaming output truncated to the last 5000 lines.
Iteration:  11% 1255/11250 [1:22:19<10:34:06,  3.81s/it]
Iteration:  11% 1256/11250 [1:22:23<10:41:58,  3.85s/it]
Iteration:  11% 1257/11250 [1:22:27<11:04:31,  3.99s/it]
Iteration:  11% 1258/11250 [1:22:31<10:42:54,  3.86s/it]
Iteration:  11% 1259/11250 [1:22:34<10:27:24,  3.77s/it]
Iteration:  11% 1260/11250 [1:22:39<11:13:33,  4.05s/it]
Iteration:  11% 1261/11250 [1:22:43<10:50:53,  3.91s/it]
Iteration:  11% 1262/11250 [1:22:46<10:36:20,  3.82s/it]
Iteration:  11% 1263/11250 [1:22:50<10:40:34,  3.85s/it]
Iteration:  11% 1264/11250 [1:22:55<11:10:26,  4.03s/it]
Iteration:  11% 1265/11250 [1:22:58<10:49:24,  3.90s/it]
Iteration:  11% 1266/11250 [1:23:02<10:30:37,  3.79s/it]
Iteration:  11% 1267/11250 [1:23:06<11:12:20,  4.04s/it]
Iteration:  11% 1268/11250 [1:23:10<10:54:15,  3.93s/it]
Iteration:  11% 1269/11250 [1:23:14<10:38:59,  3.84s/it]
Iteration:  11% 1270/11250 [1:23:18<10:39:08,  3.84s/it]
Iteration:  11% 1271/11250 [1:23:22<1

In [ ]:
!pip install fastNLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.7/644.7 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.3/217.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 3.8 MB/s eta 0:00:00
  Created wheel for fastNLP: filename=FastNLP-1.0.1-py3-none-any.whl size=811351 sha256=8459e24301f4316b4512712c920cdccdb59faaa8745474bac1dc9b30df42280b
  Stored in directory: /root/.cache/pip/wheels/d9/2a/3d/94e7148cb717edd3890429b716692c12460b2436b965bf999e
Successfully built fastNLP
  Attempting uninstall: rich
    Found existing installation: rich 13.9.4
    Uninstalling rich-13.9.4:
      Successfully uninstalled rich-13.9.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.41.0 requires rich<14,>=12.4.4, but you have rich 11.2.0 which is incompatible.
pymc 5.21.1 requi

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
!python train.py \
  --train_path train.json \
  --test_path test.json \
  --model Transformer \
  --seq_len 512 \
  --batch_size 8 \
  --do_test \
  --ckpt_name seqxgen_final.pt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-03-28 01:51:40.697418: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743126700.723564   49495 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743126700.731204   49495 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-28 01:51:40.756816: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
100% 90000/900

In [ ]:
import pandas as pd
import json
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Load your parquet
df = pd.read_parquet("/content/drive/MyDrive/fp/combined_df_preprocessed.parquet")

# Convert to SeqXGPT input format
output_data = []
for _, row in df.iterrows():
    tokens = word_tokenize(row["text"])
    label = int(row["label"])
    token_labels = [label] * len(tokens)
    output_data.append({"text": tokens, "labels": token_labels})

# Save to JSON
with open("/content/drive/MyDrive/fp/combined_seqxgpt_input.json", "w") as f:
    json.dump(output_data, f)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [33]:
!python train.py \
  --train_path train.json \
  --test_path combined_preprocessed_seqx_fixed.json \
  --model Transformer \
  --seq_len 512 \
  --batch_size 4 \
  --do_test \
  --skip_word_eval \
  --ckpt_name seqxgen_final.pt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-03-30 01:20:12.699747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743297613.154778   96459 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743297613.279567   96459 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-30 01:20:14.316645: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
100% 90000/900

### break up into 12

In [46]:
%cd ..
!pwd

/content/drive/MyDrive/fp
/content/drive/MyDrive/fp


In [69]:
final_df = pd.read_csv('/content/drive/MyDrive/fp/combined_data.csv')

In [63]:
final_df['concatid'] = final_df['label'].astype('str') + final_df['source']

In [65]:
df_sampled, _ = train_test_split(
    final_df,
    train_size=100_000,
    stratify=final_df['concatid'],
    random_state=42
)

train_df_final, test_df_final = train_test_split(
    df_sampled,
    test_size=0.2,
    stratify=df_sampled['concatid'],
    random_state=42
)

In [66]:
train_df_final.to_csv('train_df_final.csv',index=False)
test_df_final.to_csv('test_df_final.csv',index=False)

In [68]:
len(train_df_final),len(test_df_final)

(80000, 20000)

In [38]:
import json

input_path = "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/combined_preprocessed_seqx_fixed.json"
output_dir = "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/"
num_chunks = 12

# Read all lines first
with open(input_path, "r") as f:
    lines = f.readlines()

chunk_size = len(lines) // num_chunks

for i in range(num_chunks):
    chunk_lines = lines[i * chunk_size : (i + 1) * chunk_size] if i < num_chunks - 1 else lines[i * chunk_size :]
    chunk_path = f"{output_dir}test_chunk_{i+1}.json"
    with open(chunk_path, "w") as f:
        f.writelines(chunk_lines)
    print(f"✅ Saved {len(chunk_lines)} lines to {chunk_path}")


✅ Saved 64305 lines to /content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/test_chunk_1.json
✅ Saved 64305 lines to /content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/test_chunk_2.json
✅ Saved 64305 lines to /content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/test_chunk_3.json
✅ Saved 64305 lines to /content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/test_chunk_4.json
✅ Saved 64305 lines to /content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/test_chunk_5.json
✅ Saved 64305 lines to /content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/test_chunk_6.json
✅ Saved 64305 lines to /content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/test_chunk_7.json
✅ Saved 64305 lines to /content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/test_chunk_8.json
✅ Saved 64305 lines to /content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/test_chunk_9.json
✅ Saved 64305 lines to /content/drive/MyDrive/fp/SeqXGPTModel/Se

In [19]:
import json

input_path = "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/test_set_final_seqx_ready.json"
output_path_1 = "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/test_split_1.json"
output_path_2 = "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/test_split_2.json"

with open(input_path, "r") as f:
    lines = f.readlines()

half = len(lines) // 2
split1 = lines[:half]
split2 = lines[half:]

with open(output_path_1, "w") as f1:
    f1.writelines(split1)

with open(output_path_2, "w") as f2:
    f2.writelines(split2)

print(f"✅ Split complete! {len(split1)} lines in part 1, {len(split2)} in part 2.")


✅ Split complete! 77167 lines in part 1, 77167 in part 2.


In [20]:
!python train.py \
  --train_path train.json \
  --test_path test_split_1.json \
  --model Transformer \
  --seq_len 512 \
  --batch_size 8 \
  --do_test \
  --skip_word_eval \
  --ckpt_name seqxgen_final.pt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-04-01 02:25:45.153123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743474345.172017   13240 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743474345.176477   13240 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-01 02:25:45.196185: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
100% 90000/900

In [24]:
import pandas as pd

df = pd.read_json("/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/test_split_1.json", lines=True)
print(df["label"].value_counts())


label
0    39069
1    38098
Name: count, dtype: int64


In [25]:
import json
from nltk.tokenize import word_tokenize

file_path = "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/test_split_1.json"  # change this if needed

too_long = 0
total = 0

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        text = item["text"]
        total_words = len(word_tokenize(text))
        if total_words > 512:
            too_long += 1
        total += 1

print(f"Too long samples: {too_long} / {total}")


Too long samples: 910 / 77167


In [26]:
print("Sample predictions:", pred_labels[:10])
print("Unique predicted labels:", set(pred_labels))


NameError: name 'pred_labels' is not defined

In [ ]:
from nltk.tokenize import sent_tokenize

too_long = 0
for text in df["text"]:
    total_words = len(text.split())
    if total_words > 512:
        too_long += 1

print(f"Too long samples: {too_long} / {len(df)}")


In [21]:
!python train.py \
  --train_path train.json \
  --test_path test_split_2.json \
  --model Transformer \
  --seq_len 512 \
  --batch_size 8 \
  --do_test \
  --skip_word_eval \
  --ckpt_name seqxgen_final.pt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-04-01 03:01:31.790097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743476491.834834   21875 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743476491.844774   21875 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-01 03:01:31.901115: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
100% 90000/900

In [10]:
import json

In [16]:
test_df = pd.read_parquet("/content/drive/MyDrive/fp/test_set_final.parquet")
test_df['label'] = test_df['source']
test_df = test_df[['text','label']]

test_df

,text,label
99236,We still have a dominant foot as well. Think a...,0
677686,"""Only 8 more days until I turn 20! My iPod see...",1
348424,The AC button controls the air conditioning co...,1
392131,"""Yup, you're right, it's crazy that there's st...",1
162682,The worst is after being out all day on rough ...,0
...,...,...
45509,"The ink doesn't dry, because the little ball a...",0
760836,What’s the best advice you’ve ever received? S...,1
620555,"@""tchvinkle thank you so much, bestie!""",1
283444,They ask for race and gender info mainly for r...,1


In [17]:
output_path = "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/test_set_final_seqx_ready.json"
with open(output_path, "w") as fout:
    for idx, row in test_df.iterrows():
        text = str(row["text"])
        label = int(row["label"])
        label_int = label  # You can remap if needed
        prompt_len = 0

        # Dummy token values: 4 x 10 matrix of zeros
        ll_tokens_list = [[0.0] * 10 for _ in range(4)]
        begin_idx_list = [0] * 4

        item = {
            "text": text,
            "label": label,
            "label_int": label_int,
            "prompt_len": prompt_len,
            "ll_tokens_list": ll_tokens_list,
            "begin_idx_list": begin_idx_list
        }

        fout.write(json.dumps(item) + "\n")

print("✅ Converted test_set_final.parquet to SeqXGPT-compatible JSON format.")


✅ Converted test_set_final.parquet to SeqXGPT-compatible JSON format.


In [10]:
!python train.py \
  --test_path tiny_split_1.json \
  --train_path train.json \
  --model Transformer \
  --do_test \
  --ckpt_name seqxgen_final.pt \
  --batch_size 2 \
  --skip_word_eval


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-04-05 00:47:33.561645: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743814053.586006    2246 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743814053.593186    2246 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-05 00:47:33.616758: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
100% 90000/900

In [12]:
import json
from collections import Counter

with open("predictions_debug_20250405_004749.json") as f:
    preds = json.load(f)

print("Sample prediction (first entry):", preds[0])
print("Total predictions:", len(preds))

# Flatten the list of lists
flat_preds = [label for sent in preds for label in sent if label != -1]  # skip padding

print("Flattened prediction distribution:", Counter(flat_preds))


Sample prediction (first entry): [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -

In [24]:
import json

train_path = "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/train.json"
test_path = "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/tiny_split_1.json"

# Load the JSONL files
def load_texts(path):
    with open(path, 'r', encoding='utf-8') as f:
        return [json.loads(line)['text'] for line in f]

train_texts = load_texts(train_path)
test_texts = load_texts(test_path)

# Convert to sets for easy comparison
train_set = set(train_texts)
test_set = set(test_texts)

# Find overlaps
overlap = train_set.intersection(test_set)

print(f"Total train samples: {len(train_texts)}")
print(f"Total test samples: {len(test_texts)}")
print(f"Number of overlapping text entries: {len(overlap)}")

# Optional: print a few overlaps
if len(overlap) > 0:
    print("\nExample overlapping texts:")
    for i, text in enumerate(list(overlap)[:5]):
        print(f"\nOverlap {i+1}:\n{text[:300]}...")  # Print first 300 chars


Total train samples: 90000
Total test samples: 10
Number of overlapping text entries: 3

Example overlapping texts:

Overlap 1:
"Yup, you're right, it's crazy that there's still a small number of polio cases around the world. The thing is, we're not talking about the US or Western countries anymore. The remaining cases are mainly in a few countries where the vaccine is hard to deliver due to conflict or infrastructure issues...

Overlap 2:
We still have a dominant foot as well. Think about it, when you go to kick a football or soccer ball, which foot do you use? Most people would pick one or the other, I mean I'd say I'm right footed as I can't kick a football or soccer ball with my left foot worth beans....

Overlap 3:
So, dinosaurs themselves emerged during the Middle to Late Triassic period, around 230 million years ago. The first dinosaurs were small, meat-eating species like Eoraptor and Herrerasaurus.

The Triassic period ended with a mass extinction event 201 million years ago, 

In [28]:
import pandas as pd

test_df = pd.read_parquet("/content/drive/MyDrive/fp/train_set_final.parquet")
test_df['label'] = test_df['source']
test_df = test_df[['text','label']]
test_df['label'] = test_df['label'].astype(int)

test_df

,text,label
371533,SAP is a big deal because it’s one of the lead...,1
475497,Creating a drug to make people lose weight is ...,1
212345,“Semantic satiation” is the phrase used to des...,0
287410,Celebrity nudes are typically leaked through h...,1
666934,"""@kirstiealley My little chihuahua and I are s...",1
...,...,...
193502,Any system that uses usernames and passwords h...,0
186701,We the OP means is if you were going to copy a...,0
33726,You are entering your first R.E.M. cycle. This...,0
410038,America has a massive cultural influence due t...,1


In [29]:
df = test_df

In [30]:
import pandas as pd
import json
from tqdm import tqdm


output_path = "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/SeqXGPT_formatted_train.json"

with open(output_path, "w", encoding="utf-8") as f:
    for _, row in tqdm(df.iterrows(), total=len(df)):
        text = row["text"]
        label = row["label"]

        # Ensure label is a list
        if isinstance(label, int):
            label = [label]

        token_count = len(label)

        # Dummy ll_tokens_list: 3 models, each with `token_count` token features of dim 1 (can be any float)
        ll_tokens_list = [[ [0.1] for _ in range(token_count) ] for _ in range(3)]

        sample = {
            "text": text,
            "label": label,
            "prompt_len": 0,
            "ll_tokens_list": ll_tokens_list,
            "begin_idx_list": [0, 0, 0]
        }

        f.write(json.dumps(sample, ensure_ascii=False) + "\n")

print("✅ Conversion complete. Saved to:", output_path)


100%|██████████| 617334/617334 [00:48<00:00, 12707.16it/s]

✅ Conversion complete. Saved to: /content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/SeqXGPT_formatted_train.json


In [33]:
import json
import random

input_path = "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/SeqXGPT_formatted_train.json"
output_path = "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/50k_train.json"

# Load and shuffle
with open(input_path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

sampled_data = random.sample(data, 50000)

# Save the sampled subset
with open(output_path, "w", encoding="utf-8") as f:
    for item in sampled_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"✅ Sampled 50,000 entries saved to: {output_path}")


✅ Sampled 50,000 entries saved to: /content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/50k_train.json


In [36]:
import json

input_path = "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/50k_train.json"
output_path = "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/50k_train_fixed.json"

with open(input_path, "r") as fin, open(output_path, "w") as fout:
    for line in fin:
        sample = json.loads(line)

        # Flatten each sublist in ll_tokens_list if it's triple-nested
        flattened = []
        for token_list in sample["ll_tokens_list"]:
            if isinstance(token_list, list) and isinstance(token_list[0], list):
                flattened.append([float(x) for x in token_list[0]])
            else:
                flattened.append([float(x) for x in token_list])

        sample["ll_tokens_list"] = flattened
        fout.write(json.dumps(sample) + "\n")

print(f"✅ Saved fixed file to: {output_path}")


✅ Saved fixed file to: /content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/50k_train_fixed.json


In [35]:
import json

# Path to your sampled training file
file_path = "/content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/50k_train.json"

# Load the first sample
with open(file_path, "r") as f:
    first_sample = json.loads(f.readline())

# Print the keys and types
print("Sample keys and types:")
for key, value in first_sample.items():
    print(f"{key}: type = {type(value)}")
    if isinstance(value, list):
        print(f"  - Length: {len(value)}")
        if all(isinstance(v, list) for v in value):
            print(f"  - Inner list length: {[len(v) for v in value[:3]]} (showing first 3)")

# Optionally, show a snippet of each key's value
print("\nSample content preview:")
for key, value in first_sample.items():
    if isinstance(value, list):
        print(f"{key}: {str(value)[:200]}...")
    else:
        print(f"{key}: {value}")


Sample keys and types:
text: type = <class 'str'>
label: type = <class 'list'>
  - Length: 1
prompt_len: type = <class 'int'>
ll_tokens_list: type = <class 'list'>
  - Length: 3
  - Inner list length: [1, 1, 1] (showing first 3)
begin_idx_list: type = <class 'list'>
  - Length: 3

Sample content preview:
text: It tracks mouse movement and the timing of clicking the box.  If a program is too fast, too slow, too straight, or a bunch of other things it doesn't "pass".
label: [0]...
prompt_len: 0
ll_tokens_list: [[[0.1]], [[0.1]], [[0.1]]]...
begin_idx_list: [0, 0, 0]...


In [1]:
!python train.py \
  --train_path /content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/final_train.json \
  --test_path /content/drive/MyDrive/fp/SeqXGPTModel/SeqXGPT/SeqXGPT/SeqXGPT/test_split_1.json \
  --model Transformer \
  --seq_len 512 \
  --batch_size 16 \
  --num_train_epochs 3 \
  --lr 5e-5 \
  --ckpt_name retrained_sample_50k.pt


python3: can't open file '/content/train.py': [Errno 2] No such file or directory


In [41]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet('/content/drive/MyDrive/fp/train_set_final.parquet')

# Show the first 5 rows
print("First 5 rows:")
print(df.head())


First 5 rows:
                                                     text    label source
371533  SAP is a big deal because it’s one of the lead...   reddit      1
475497  Creating a drug to make people lose weight is ...   reddit      1
212345  “Semantic satiation” is the phrase used to des...   reddit      0
287410  Celebrity nudes are typically leaked through h...   reddit      1
666934  "@kirstiealley My little chihuahua and I are s...  twitter      1


In [42]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split

# Confirm label is integer, else map to int
if not pd.api.types.is_integer_dtype(df["source"]):
    unique_labels = sorted(df["source"].dropna().unique())
    label_to_int = {label: idx for idx, label in enumerate(unique_labels)}
    df["label_int"] = df["source"].map(label_to_int)
else:
    df["label_int"] = df["source"]

# Add required dummy fields
df["prompt_len"] = 0
df["begin_idx_list"] = [[0, 0, 0, 0]] * len(df)

# Create dummy ll_tokens_list (4 channels of 10-dim zero vectors)
def make_dummy_ll_tokens():
    return [[0.0] * 10 for _ in range(4)]

df["ll_tokens_list"] = [make_dummy_ll_tokens() for _ in range(len(df))]

# Split the dataset
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Save to JSONL
def save_jsonl(df, filepath):
    with open(filepath, "w", encoding="utf-8") as f:
        for _, row in df.iterrows():
            obj = {
                "text": row["text"],
                "label": int(row["label_int"]),
                "prompt_len": int(row["prompt_len"]),
                "begin_idx_list": row["begin_idx_list"],
                "ll_tokens_list": row["ll_tokens_list"]
            }
            f.write(json.dumps(obj, ensure_ascii=False) + "\n")

save_jsonl(df, "/content/drive/MyDrive/fp/final_train.json")

print("✅ Finished writing train.json and test.json")


✅ Finished writing train.json and test.json
